# Workflow Interface 1001: Workspace Creation from Jupyter Notebook

This tutorial demonstrates the methodology to convert a Federated Learning experiment developed in Jupyter Notebook into a Workspace that can be deployed using Aggregator Based Workflow

OpenFL experimental Workflow Interface enables the user to simulate a Federated Learning experiment using **LocalRuntime**. Once the simulation is ready, the methodology described in this tutorial enables the user to convert this experiment into an OpenFL workspace that can be deployed using the Aggregator-Based-Workflow

##### High Level Overview of Methodology
1. User annotates the relevant cells of the Jupyter notebook with `#| export` directive
2. We then Leverage `nbdev` functionality to export these annotated cells of Jupyter notebook into a Python script
3. Utilize OpenFL experimental module `WorkspaceExport` to convert the Python script into a OpenFL workspace
4. User can utilize the experimental `fx` commands to deploy and run the federation seamlessly


The methodology is described using an existing [OpenFL Watermarking Tutorial](https://github.com/securefederatedai/openfl/blob/develop/openfl-tutorials/experimental/Workflow_Interface_301_MNIST_Watermarking.ipynb). Let's get started !



# Getting Started

Initially, we start by specifying the module where cells marked with the `#| export` directive will be automatically exported. 

In the following cell, `#| default_exp experiment `indicates that the exported file will be named 'experiment'. This name can be modified based on user's requirement & preferences

In [ ]:
#| default_exp experiment

Once we have specified the name of the module, subsequent cells of the notebook need to be *appended* by the `#| export` directive as shown below. User should ensure that *all* the notebook functionality required in the Federated Learning experiment is included in this directive

We start by installing OpenFL and dependencies of the workflow interface 
> These dependencies are required to be exported and become the requirements for the Federated Learning Workspace 

In [ ]:
#| export

!pip install git+https://github.com/securefederatedai/openfl.git
!pip install -r workflow_interface_requirements.txt

!pip install matplotlib
!pip install torch
!pip install torchvision
!pip install git+https://github.com/pyviz-topics/imagen.git@master
!pip install holoviews==1.15.4


# Uncomment this if running in Google Colab
#!pip install -r https://raw.githubusercontent.com/intel/openfl/develop/openfl-tutorials/experimental/workflow_interface_requirements.txt
#import os
#os.environ["USERNAME"] = "colab"

We now define our dataloaders, model, optimizer, and some helper functions like we would for any other deep learning experiment 

> This cell and all the subsequent cells are important ingredients of the Federated Learning experiment and therefore annotated with the `#| export` directive

In [ ]:
# | export

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import numpy as np
import random
import pathlib
import os
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image as Image
import imagen as ig
import numbergen as ng
import os

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

# MNIST Train and Test datasets
mnist_train = torchvision.datasets.MNIST(
    "./files/",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        ]
    ),
)

mnist_test = torchvision.datasets.MNIST(
    "./files/",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        ]
    ),
)


class Net(nn.Module):
    def __init__(self, dropout=0.0):
        super(Net, self).__init__()
        self.dropout = dropout
        self.block = nn.Sequential(
            nn.Conv2d(1, 32, 2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 2),
            nn.ReLU(),
        )
        self.fc1 = nn.Linear(128 * 5**2, 200)
        self.fc2 = nn.Linear(200, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.dropout(x)
        out = self.block(x)
        out = out.view(-1, 128 * 5**2)
        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return F.log_softmax(out, 1)


def inference(network, test_loader):
    network.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    accuracy = float(correct / len(test_loader.dataset))
    return accuracy


def train_model(model, optimizer, data_loader, entity, round_number, log=False):
    # Helper function to train the model
    train_loss = 0
    log_interval = 20
    model.train()
    for batch_idx, (X, y) in enumerate(data_loader):
        optimizer.zero_grad()

        output = model(X)
        loss = F.nll_loss(output, y)
        loss.backward()

        optimizer.step()

        train_loss += loss.item() * len(X)
        if batch_idx % log_interval == 0 and log:
            print("{:<20} Train Epoch: {:<3} [{:<3}/{:<4} ({:<.0f}%)] Loss: {:<.6f}".format(
                    entity,
                    round_number,
                    batch_idx * len(X),
                    len(data_loader.dataset),
                    100.0 * batch_idx / len(data_loader),
                    loss.item(),
                )
            )
    train_loss /= len(data_loader.dataset)
    return train_loss

Next we define the dataset required for watermarking

In [ ]:
#| export

watermark_dir = "./files/watermark-dataset/MWAFFLE/"


def generate_watermark(
    x_size=28, y_size=28, num_class=10, num_samples_per_class=10, img_dir=watermark_dir
):
    """
    Generate Watermark by superimposing a pattern on noisy background.

    Parameters
    ----------
    x_size: x dimension of the image
    y_size: y dimension of the image
    num_class: number of classes in the original dataset
    num_samples_per_class: number of samples to be generated per class
    img_dir: directory for saving watermark dataset

    Reference
    ---------
    WAFFLE: Watermarking in Federated Learning (https://arxiv.org/abs/2008.07298)

    """
    x_pattern = int(x_size * 2 / 3.0 - 1)
    y_pattern = int(y_size * 2 / 3.0 - 1)

    np.random.seed(0)
    for cls in range(num_class):
        patterns = []
        random_seed = 10 + cls
        patterns.append(
            ig.Line(
                xdensity=x_pattern,
                ydensity=y_pattern,
                thickness=0.001,
                orientation=np.pi * ng.UniformRandom(seed=random_seed),
                x=ng.UniformRandom(seed=random_seed) - 0.5,
                y=ng.UniformRandom(seed=random_seed) - 0.5,
                scale=0.8,
            )
        )
        patterns.append(
            ig.Arc(
                xdensity=x_pattern,
                ydensity=y_pattern,
                thickness=0.001,
                orientation=np.pi * ng.UniformRandom(seed=random_seed),
                x=ng.UniformRandom(seed=random_seed) - 0.5,
                y=ng.UniformRandom(seed=random_seed) - 0.5,
                size=0.33,
            )
        )

        pat = np.zeros((x_pattern, y_pattern))
        for i in range(6):
            j = np.random.randint(len(patterns))
            pat += patterns[j]()
        res = pat > 0.5
        pat = res.astype(int)

        x_offset = np.random.randint(x_size - x_pattern + 1)
        y_offset = np.random.randint(y_size - y_pattern + 1)

        for i in range(num_samples_per_class):
            base = np.random.rand(x_size, y_size)
            # base = np.zeros((x_input, y_input))
            base[
                x_offset : x_offset + pat.shape[0],
                y_offset : y_offset + pat.shape[1],
            ] += pat
            d = np.ones((x_size, x_size))
            img = np.minimum(base, d)
            if not os.path.exists(img_dir + str(cls) + "/"):
                os.makedirs(img_dir + str(cls) + "/")
            plt.imsave(
                img_dir + str(cls) + "/wm_" + str(i + 1) + ".png",
                img,
                cmap=matplotlib.cm.gray,
            )


# If the Watermark dataset does not exist, generate and save the Watermark images
watermark_path = pathlib.Path(watermark_dir)
if watermark_path.exists() and watermark_path.is_dir():
    print(
        f"Watermark dataset already exists at: {watermark_path}. Proceeding to next step ... "
    )
    pass
else:
    print(f"Generating Watermark dataset... ")
    generate_watermark()


class WatermarkDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, label_dir=None, transforms=None):
        self.images_dir = os.path.abspath(images_dir)
        self.image_paths = [
            os.path.join(self.images_dir, d) for d in os.listdir(self.images_dir)
        ]
        self.label_paths = label_dir
        self.transform = transforms
        temp = []

        # Recursively counting total number of images in the directory
        for image_path in self.image_paths:
            for path in os.walk(image_path):
                if len(path) <= 1:
                    continue
                path = path[2]
                for im_n in [image_path + "/" + p for p in path]:
                    temp.append(im_n)
        self.image_paths = temp

        if len(self.image_paths) == 0:
            raise Exception(f"No file(s) found under {images_dir}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = Image.open(image_filepath)
        image = image.convert("RGB")
        image = self.transform(image)
        label = int(image_filepath.split("/")[-2])

        return image, label


def get_watermark_transforms():
    return torchvision.transforms.Compose(
        [
            torchvision.transforms.Grayscale(),
            torchvision.transforms.Resize(28),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)),  # Normalize
        ]
    )


watermark_data = WatermarkDataset(
    images_dir=watermark_dir,
    transforms=get_watermark_transforms(),
)

# Set display_watermark to True to display the Watermark dataset
display_watermark = False
if display_watermark:
    # Inspect and plot the Watermark Images
    wm_images = np.empty((100, 28, 28))
    wm_labels = np.empty([100, 1], dtype=int)

    for i in range(len(watermark_data)):
        img, label = watermark_data[i]
        wm_labels[label * 10 + i % 10] = label
        wm_images[label * 10 + i % 10, :, :] = img.numpy()

    fig = plt.figure(figsize=(120, 120))
    for i in range(100):
        plt.subplot(10, 10, i + 1)
        plt.imshow(wm_images[i], interpolation="none")
        plt.title("Label: {}".format(wm_labels[i]), fontsize=80)

Next we import the `FLSpec`, `LocalRuntime`, placement decorators (`aggregator/collaborator`)

In [ ]:
#| export

from copy import deepcopy

from openfl.experimental.interface import FLSpec, Aggregator, Collaborator
from openfl.experimental.runtime import LocalRuntime
from openfl.experimental.placement import aggregator, collaborator

def FedAvg(agg_model, models, weights=None):
    state_dicts = [model.state_dict() for model in models]
    state_dict = agg_model.state_dict()
    for key in models[0].state_dict():
        state_dict[key] = torch.from_numpy(np.average([state[key].numpy() for state in state_dicts],
                                                      axis=0, 
                                                      weights=weights))
        
    agg_model.load_state_dict(state_dict)
    return agg_model

Let us now define the Workflow for Watermark embedding.

In [ ]:
#| export

class FederatedFlow_MNIST_Watermarking(FLSpec):
    """
    This Flow demonstrates Watermarking on a Deep Learning Model in Federated Learning
    Ref: WAFFLE: Watermarking in Federated Learning (https://arxiv.org/abs/2008.07298)
    """

    def __init__(
        self,
        model=None,
        optimizer=None,
        watermark_pretrain_optimizer=None,
        watermark_retrain_optimizer=None,
        round_number=0,
        **kwargs,
    ):
        super().__init__(**kwargs)

        if model is not None:
            self.model = model
            self.optimizer = optimizer
            self.watermark_pretrain_optimizer = watermark_pretrain_optimizer
            self.watermark_retrain_optimizer = watermark_retrain_optimizer
        else:
            self.model = Net()
            self.optimizer = optim.SGD(
                self.model.parameters(), lr=learning_rate, momentum=momentum
            )
            self.watermark_pretrain_optimizer = optim.SGD(
                self.model.parameters(),
                lr=watermark_pretrain_learning_rate,
                momentum=watermark_pretrain_momentum,
                weight_decay=watermark_pretrain_weight_decay,
            )
            self.watermark_retrain_optimizer = optim.SGD(
                self.model.parameters(), lr=watermark_retrain_learning_rate
            )
        self.round_number = round_number
        self.watermark_pretraining_completed = False

    @aggregator
    def start(self):
        """
        This is the start of the Flow.
        """

        print(f"<Agg>: Start of flow ... ")
        self.collaborators = self.runtime.collaborators

        # Randomly select a fraction of actual collaborator every round
        fraction = 0.5
        if int(fraction * len(self.collaborators)) < 1:
            raise Exception(
                f"Cannot run training with {fraction*100}% selected collaborators out of {len(self.collaborators)} Collaborators. Atleast one collaborator is required to run the training"
            )
        self.subset_collaborators = random.sample(
            self.collaborators, int(fraction * (len(self.collaborators)))
        )

        self.next(self.watermark_pretrain)

    @aggregator
    def watermark_pretrain(self):
        """
        Pre-Train the Model before starting Federated Learning.
        """
        if not self.watermark_pretraining_completed:

            print("<Agg>: Performing Watermark Pre-training")

            for i in range(self.pretrain_epochs):

                watermark_pretrain_loss = train_model(
                    self.model,
                    self.watermark_pretrain_optimizer,
                    self.watermark_data_loader,
                    "<Agg>:",
                    i,
                    log=False,
                )
                watermark_pretrain_validation_score = inference(
                    self.model, self.watermark_data_loader
                )

                print(
                    "<Agg>: Watermark Pretraining: Round: {:<3} Loss: {:<.6f} Acc: {:<.6f}".format(
                        i,
                        watermark_pretrain_loss,
                        watermark_pretrain_validation_score,
                    )
                )

            self.watermark_pretraining_completed = True

        self.next(
            self.aggregated_model_validation,
            foreach="subset_collaborators",
            exclude=["watermark_pretrain_optimizer", "watermark_retrain_optimizer"],
        )

    @collaborator
    def aggregated_model_validation(self):
        """
        Perform Aggregated Model validation on Collaborators.
        """
        self.agg_validation_score = inference(self.model, self.test_loader)
        print(
            f"<Collab: {self.input}> Aggregated Model validation score = {self.agg_validation_score}"
        )

        self.next(self.train)

    @collaborator
    def train(self):
        """
        Train model on Local collab dataset.

        """
        print("<Collab>: Performing Model Training on Local dataset ... ")

        self.optimizer = optim.SGD(
            self.model.parameters(), lr=learning_rate, momentum=momentum
        )

        self.loss = train_model(
            self.model,
            self.optimizer,
            self.train_loader,
            "<Collab: {:<20}".format(self.input + ">"),
            self.round_number if self.round_number is not None else 0,
            log=True,
        )

        self.next(self.local_model_validation)

    @collaborator
    def local_model_validation(self):
        """
        Validate locally trained model.

        """
        self.local_validation_score = inference(self.model, self.test_loader)
        print(
            f"<Collab: {self.input}> Local model validation score = {self.local_validation_score}"
        )
        self.next(self.join)

    @aggregator
    def join(self, inputs):
        """
        Model aggregation step.
        """

        self.average_loss = sum(input.loss for input in inputs) / len(inputs)
        self.aggregated_model_accuracy = sum(
            input.agg_validation_score for input in inputs
        ) / len(inputs)
        self.local_model_accuracy = sum(
            input.local_validation_score for input in inputs
        ) / len(inputs)

        print(f"<Agg>: Joining models from collaborators...")

        print(
            f"   Aggregated model validation score = {self.aggregated_model_accuracy}"
        )
        print(f"   Average training loss = {self.average_loss}")
        print(f"   Average local model validation values = {self.local_model_accuracy}")

        self.model = FedAvg(self.model, [input.model for input in inputs])

        self.next(self.watermark_retrain)

    @aggregator
    def watermark_retrain(self):
        """
        Retrain the aggregated model.

        """
        print("<Agg>: Performing Watermark Retraining ... ")
        self.watermark_retrain_optimizer = optim.SGD(
            self.model.parameters(), lr=watermark_retrain_learning_rate
        )

        retrain_round = 0

        # Perform re-training until (accuracy >= acc_threshold) or (retrain_round > number of retrain_epochs)
        self.watermark_retrain_validation_score = inference(
            self.model, self.watermark_data_loader
        )
        while (
            self.watermark_retrain_validation_score < self.watermark_acc_threshold
        ) and (retrain_round < self.retrain_epochs):
            self.watermark_retrain_train_loss = train_model(
                self.model,
                self.watermark_retrain_optimizer,
                self.watermark_data_loader,
                "<Agg>",
                retrain_round,
                log=False,
            )
            self.watermark_retrain_validation_score = inference(
                self.model, self.watermark_data_loader
            )

            print(
                "<Agg>: Watermark Retraining: Train Epoch: {:<3} Retrain Round: {:<3} Loss: {:<.6f}, Acc: {:<.6f}".format(
                    self.round_number,
                    retrain_round,
                    self.watermark_retrain_train_loss,
                    self.watermark_retrain_validation_score,
                )
            )

            retrain_round += 1

        self.next(self.end)

    @aggregator
    def end(self):
        """
        This is the last step in the Flow.

        """
        print(f"This is the end of the flow")

We now initialize certain attributes of the Flow, simulation parameters (seed, batch-sizes, optimizer parameters) and create the `LocalRuntime`

> NOTE: Aggregator based workflow requires a `FederatedRuntime`. In this methodology `FederatedRuntime` is created automatically and it's usage is transparent to the user

In [ ]:
#| export

# Set random seed
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.enabled = False

# Batch sizes
batch_size_train = 64
batch_size_test = 64
batch_size_watermark = 50

# MNIST parameters
learning_rate = 5e-2
momentum = 5e-1
log_interval = 20

# Watermarking parameters
watermark_pretrain_learning_rate = 1e-1
watermark_pretrain_momentum = 5e-1
watermark_pretrain_weight_decay = 5e-05
watermark_retrain_learning_rate = 5e-3

You'll notice in the `FederatedFlow_MNIST_Watermarking` definition above that there were certain attributes that the flow was not initialized with, namely the `train_loader` and `test_loader` for each of the collaborators. These are **private_attributes** of the particular participant and (as the name suggests) are accessible ONLY to the particular participant's through its task. Additionally these private attributes are always filtered out of the current state when transferring from collaborator to aggregator, and vice versa.

Users can directly specify a collaborator's private attributes via `collaborator.private_attributes` which is a dictionary where key is name of the attribute and value is the object that is made accessible to collaborator.

For more detailed information on specifying these private attributes, please refer to the first quick start [notebook](https://github.com/securefederatedai/openfl/blob/develop/openfl-tutorials/experimental/Workflow_Interface_101_MNIST.ipynb)

In [ ]:
#| export

# Setup Aggregator with private attributes
aggregator = Aggregator()
aggregator.private_attributes = {
    "watermark_data_loader": torch.utils.data.DataLoader(
        watermark_data, batch_size=batch_size_watermark, shuffle=True
    ),
    "pretrain_epochs": 25,
    "retrain_epochs": 25,
    "watermark_acc_threshold": 0.98,
    "watermark_pretraining_completed": False,
}

# Setup Collaborators with private attributes
collaborator_names = ['Portland', 'Seattle', 'Chandler','Bangalore']
print(f"Creating collaborators {collaborator_names}")
collaborators = [Collaborator(name=name) for name in collaborator_names]

for idx, collaborator in enumerate(collaborators):
    local_train = deepcopy(mnist_train)
    local_test = deepcopy(mnist_test)
    local_train.data = mnist_train.data[idx :: len(collaborators)]
    local_train.targets = mnist_train.targets[idx :: len(collaborators)]
    local_test.data = mnist_test.data[idx :: len(collaborators)]
    local_test.targets = mnist_test.targets[idx :: len(collaborators)]
    collaborator.private_attributes = {
        "train_loader": torch.utils.data.DataLoader(
            local_train, batch_size=batch_size_train, shuffle=True
        ),
        "test_loader": torch.utils.data.DataLoader(
            local_test, batch_size=batch_size_train, shuffle=True
        ),
    }

local_runtime = LocalRuntime(aggregator=aggregator, collaborators=collaborators)
print(f"Local runtime collaborators = {local_runtime.collaborators}")

### Alternate method to specify private attributes

There is another way users can initialize private attributes in which private attributes need to be set using a (user defined) callback function while instantiating the participant. The callback function returns the private attributes (`train_loader` & `test_loader`) in form of a dictionary where the key is the attribute name, and the value is the object that will be made accessible to that participant's task.

In the following example callback function, `callable_to_initialize_collaborator_private_attributes`, returns the private attributes `train_loader` and `test_loader` of the collaborator.

Detailed information on specifying private attributes using callback function is provided in this [documentation](https://github.com/securefederatedai/openfl/blob/develop/docs/about/features_index/workflowinterface.rst). 

>To use this method, uncomment the following cell and comment out the previous cell which initializes private attributes directly.

In [ ]:
# #| export

# def callable_to_initialize_aggregator_private_attributes(watermark_data, batch_size):
#     return {
#         "watermark_data_loader": torch.utils.data.DataLoader(
#             watermark_data, batch_size=batch_size, shuffle=True
#         ),
#         "pretrain_epochs": 25,
#         "retrain_epochs": 25,
#         "watermark_acc_threshold": 0.98,
#     }

# # Setup Aggregator private attributes via callable function
# aggregator = Aggregator(
#         name="agg",
#         private_attributes_callable=callable_to_initialize_aggregator_private_attributes,
#         watermark_data=watermark_data,
#         batch_size=batch_size_watermark,
#     )

# collaborator_names = [
#     "Portland",
#     "Seattle",
#     "Chandler",
#     "Bangalore",
#     "New Delhi",
# ]
# n_collaborators = len(collaborator_names)

# def callable_to_initialize_collaborator_private_attributes(index, n_collaborators, batch_size, train_dataset, test_dataset):
#     train = deepcopy(train_dataset)
#     test = deepcopy(test_dataset)
#     train.data = train_dataset.data[index::n_collaborators]
#     train.targets = train_dataset.targets[index::n_collaborators]
#     test.data = test_dataset.data[index::n_collaborators]
#     test.targets = test_dataset.targets[index::n_collaborators]

#     return {
#         "train_loader": torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True),
#         "test_loader": torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True),
#     }

# # Setup Collaborators private attributes via callable function
# collaborators = []
# for idx, collaborator_name in enumerate(collaborator_names):
#     collaborators.append(
#         Collaborator(
#             name=collaborator_name, num_cpus=0, num_gpus=0,
#             private_attributes_callable=callable_to_initialize_collaborator_private_attributes,
#             index=idx, n_collaborators=n_collaborators,
#             train_dataset=mnist_train, test_dataset=mnist_test, batch_size=64
#         )
#     )

# local_runtime = LocalRuntime(aggregator=aggregator, collaborators=collaborators, backend="ray")
# print(f"Local runtime collaborators = {local_runtime.collaborators}")

>NOTE: If both methods for specifying private attributes are used, the private attributes will only be set by the latter method. Additionally, the code for both methods  will be included in your generated workspace.

Now that we have our flow and runtime defined, let's run the experiment! 

In [ ]:
#| export

model = Net()
optimizer = optim.SGD(
    model.parameters(), lr=learning_rate, momentum=momentum
)
watermark_pretrain_optimizer = optim.SGD(
    model.parameters(),
    lr=watermark_pretrain_learning_rate,
    momentum=watermark_pretrain_momentum,
    weight_decay=watermark_pretrain_weight_decay,
)
watermark_retrain_optimizer = optim.SGD(
    model.parameters(), lr=watermark_retrain_learning_rate
)
best_model = None
round_number = 0
top_model_accuracy = 0

flflow = FederatedFlow_MNIST_Watermarking(
    model,
    optimizer,
    watermark_pretrain_optimizer,
    watermark_retrain_optimizer,
    round_number,
    checkpoint=True,
)
flflow.runtime = local_runtime

## Workspace creation

The following cells convert the Jupyter notebook into a Python script and create a Template Workspace that can be utilized by Aggregator based Workflow
> NOTE: Only Notebook cells that were marked with `#| export` directive shall be included in this Python script

We first import `WorkspaceExport` module and execute `WorkspaceExport.export()` that converts the notebook and generates the template workspace. User is required to specify: 
1. `notebook_path`: path of the Jupyter notebook that is required to be converted
2. `output_workspace`: path where the converted workspace is stored

In [ ]:
import os
from openfl.experimental.workspace_export import WorkspaceExport

WorkspaceExport.export(
    notebook_path='./Workflow_Interface_1001_Workspace_Creation_from_JupyterNotebook.ipynb',
    output_workspace=f"/home/{os.environ['USER']}/generated-workspace"
)

## Workspace Usage

The workspace created above can be used by the Aggregator based workflow by using the `fx` commands in the following manner

**Workspace Activation and Creation**
1. Activate the experimental aggregator-based workflow:

    `fx experimental activate`

   This will create an 'experimental' directory under ~/.openfl/
3. Create a workspace using the custom template:

    `fx workspace create --prefix workspace_path --custom_template /home/$USER/generated-workspace`
4. Change to the workspace directory:

    `cd workspace_path`

**Workspace Initialization and Certification**
1. Initialize the FL plan and auto-populate the fully qualified domain name (FQDN) of the aggregator node:

    `fx plan initialize`
2. Certify the workspace:

    `fx workspace certify`
    
**Aggregator Setup and Workspace Export**
1. Run the aggregator certificate creation command:

    `fx aggregator generate-cert-request`

    `fx aggregator certify`
2. Export the workspace for collaboration:

    `fx workspace export`
    
**Collaborator Node Setup**

***On the Collaborator Node:***

1. Copy the workspace archive from the aggregator node to the collaborator nodes. Import the workspace archive:

    `fx workspace import --archive WORKSPACE.zip`
   
    `cd workspace_path`
3. Generate a collaborator certificate request:

    `fx collaborator generate-cert-request -n {COL_LABEL}`

***On the Aggregator Node (Certificate Authority):***

3. Sign the Collaborator Certificate Signing Request (CSR) Package from collaborator nodes:

    `fx collaborator certify --request-pkg /PATH/TO/col_{COL_LABEL}_to_agg_cert_request.zip`

***On the Collaborator Node:***

4. Import the signed certificate and certificate chain into the workspace:

    `fx collaborator certify --import /PATH/TO/agg_to_col_{COL_LABEL}_signed_cert.zip`
    
**Final Workspace Activation**
***On the Aggregator Node:***

1. Start the Aggregator:

    `fx aggregator start`
    
    The Aggregator is now running and waiting for Collaborators to connect.

***On the Collaborator Nodes:***

2. Run the Collaborator:

    `fx collaborator start -n {COL_LABEL}`

**Workspace Deactivation**
1. To deactivate the experimental aggregator-based workflow and switch back to original aggregator-based workflow:

    `fx experimental deactivate`

   This will remove the 'experimental' directory under ~/.openfl/
